## Convolution Neural Network (with Pytorch)

#### 

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F

import seaborn as sns
from sklearn.datasets import fetch_openml
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

In [7]:
# some vars
batch_size = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# Define transformations for dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

train_dataset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100.0%
100.0%


In [4]:
# Explore the data
print("Training dataset size:", len(train_dataset))
print("Testing dataset size:", len(test_dataset))

example_image, example_label = train_dataset[0]
print("Image shape:", example_image.shape)

# Get the number of classes
num_classes = len(train_dataset.classes)
print("Number of classes:", num_classes)

Training dataset size: 50000
Testing dataset size: 10000
Image shape: torch.Size([3, 32, 32])
Number of classes: 10


In [5]:
# defining nn architecture
class Cifar10NN(nn.Module):
    def __init__(self):
        super(Cifar10NN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)  #the layer produces 32 output feature maps (each is a result of convolving the input with a different kernel/filter).
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # Adjusted for the output size after pooling
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

        # Pooling and dropout
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))            # after this max pooling, the tensor shape becomes (batch_size, 32, 16, 16)
        x = self.pool(F.relu(self.conv2(x)))            # now (batch_size, 64, 8, 8)
        x = self.pool(F.relu(self.conv3(x)))            # now (batch_size, 128, 4, 4)

        # Flatten the output for the fully connected layers, 1-D
        x = x.view(-1, 128 * 4 * 4)

        # Fully connected layers with ReLU and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return x

In [8]:
# instantiate model, move it to gpu (if available)
model = Cifar10NN()
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.to(device)

Cifar10NN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [9]:
# training loop
num_epochs = 10


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0  # Track the loss for each epoch

    for i, data in enumerate(train_loader, 0):
        # 1. get a batch of data
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # 2. reset gradients (aside from first iteration, these have some value)
        optimizer.zero_grad()

        # 3. run the data through the model. forward pass
        y_pred = model(inputs)

        # 4. Compute the loss
        loss = cost(y_pred, labels)

        # 5. Backpropagation
        loss.backward()

        # 6. update weights
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Training finished.')

Epoch 1, Batch 100, Loss: 2.237
Epoch 1, Batch 200, Loss: 1.900
Epoch 1, Batch 300, Loss: 1.765
Epoch 1, Batch 400, Loss: 1.652
Epoch 1, Batch 500, Loss: 1.580
Epoch 1, Batch 600, Loss: 1.514
Epoch 1, Batch 700, Loss: 1.461
Epoch 2, Batch 100, Loss: 1.356
Epoch 2, Batch 200, Loss: 1.352
Epoch 2, Batch 300, Loss: 1.290
Epoch 2, Batch 400, Loss: 1.296
Epoch 2, Batch 500, Loss: 1.264
Epoch 2, Batch 600, Loss: 1.230
Epoch 2, Batch 700, Loss: 1.223
Epoch 3, Batch 100, Loss: 1.123
Epoch 3, Batch 200, Loss: 1.106
Epoch 3, Batch 300, Loss: 1.137
Epoch 3, Batch 400, Loss: 1.113
Epoch 3, Batch 500, Loss: 1.056
Epoch 3, Batch 600, Loss: 1.070
Epoch 3, Batch 700, Loss: 1.076
Epoch 4, Batch 100, Loss: 0.988
Epoch 4, Batch 200, Loss: 0.985
Epoch 4, Batch 300, Loss: 0.973
Epoch 4, Batch 400, Loss: 0.971
Epoch 4, Batch 500, Loss: 0.978
Epoch 4, Batch 600, Loss: 0.957
Epoch 4, Batch 700, Loss: 0.967
Epoch 5, Batch 100, Loss: 0.880
Epoch 5, Batch 200, Loss: 0.900
Epoch 5, Batch 300, Loss: 0.867
Epoch 5,

In [15]:
# loss plot
sns.lineplot(cm, x='epoch', y='loss', annot=True)
plt.title("CNN Loss")
plt.show()

NameError: name 'cm' is not defined

In [14]:
# evaluate model
def evaluate(data_loader):
    correct = 0
    total = 0
    with torch.inference_mode():  # Ensures inference mode is active
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Get model predictions
            _, predicted = torch.max(outputs, 1)  # Get predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy
    
print(f"Train Accuracy: {evaluate(train_loader):.2f}%")    
print(f"Test Accuracy: {evaluate(test_loader):.2f}%")    

Train Accuracy: 81.62%
Test Accuracy: 71.76%
